Introduction
============

In most presentations of Riemannian geometry, e.g. @o1983semi and
[Wikipedia](https://en.wikipedia.org/wiki/Fundamental_theorem_of_Riemannian_geometry),
the fundamental theorem of Riemannian geometry ("the miracle of
Riemannian geometry") is given: that for any semi-Riemannian manifold
there is a unique torsion-free metric connection. I assume partly
because of this and partly because the major application of Riemannian
geometry is General Relativity, connections with torsion are given
little if any attention.

It turns out we are all very familiar with a connection with torsion:
the Mercator projection. Some mathematical physics texts,
e.g. @Nakahara2003, allude to this but leave the details to the
reader. Moreover, this connection respects the metric induced from
Euclidean space.

We use [SageManifolds](http://sagemanifolds.obspm.fr) to assist with
the calculations. We hint at how this might be done more slickly in
[Haskell](https://www.haskell.org).

Colophon
--------

This is written as an [Jupyter](http://jupyter.org) notebook. In
theory, it should be possible to run it assuming you have installed at
least sage and Haskell. To publish it, I used

    jupyter-nbconvert --to markdown Mercator.ipynb
    pandoc -s Mercator.md -t markdown+lhs -o Mercator.lhs \
           --filter pandoc-citeproc --bibliography DiffGeom.bib
    BlogLiteratelyD --wplatex Mercator.lhs > Mercator.html

Not brilliant but good enough.

Warming Up With SageManifolds
=============================

Let us try a simple exercise: finding the connection coefficients of
the Levi-Civita connection for the Euclidean metric on $\mathbb{R}^2$
in polar co-ordinates.

Define the manifold.

In [710]:
%display latex

In [711]:
N = Manifold(2, 'N',r'\mathcal{N}', start_index=1)

Define a chart and frame with Cartesian co-ordinates.

In [712]:
ChartCartesianN.<x,y> = N.chart()

In [713]:
FrameCartesianN = ChartCartesianN.frame()

Define a chart and frame with polar co-ordinates.

In [714]:
ChartPolarN.<r,theta> = N.chart()

In [715]:
FramePolarN = ChartPolarN.frame()

The standard transformation from Cartesian to polar co-ordinates.

In [716]:
cartesianToPolar = ChartCartesianN.transition_map(ChartPolarN, (sqrt(x^2 + y^2), arctan(y/x)))

In [717]:
print(cartesianToPolar)

Change of coordinates from Chart (N, (x, y)) to Chart (N, (r, theta))


In [719]:
print(cartesianToPolar.display())

r = sqrt(x^2 + y^2)
theta = arctan(y/x)


In [606]:
cartesianToPolar.set_inverse(r * cos(theta), r * sin(theta))

Check of the inverse coordinate transformation:
   x == x
   y == y
   r == abs(r)
   theta == arctan(sin(theta)/cos(theta))


Now we define the metric to make the manifold Euclidean.

In [607]:
g_e = N.metric('g_e')

In [608]:
g_e[1,1], g_e[2,2] = 1, 1

We can display this in Cartesian co-ordinates.

In [609]:
print(g_e.display(FrameCartesianN))


g_e = dx*dx + dy*dy


And we can display it in polar co-ordinates

In [610]:
print(g_e.display(FramePolarN))

g_e = dr*dr + (x^2 + y^2) dtheta*dtheta


Next let us compute the Levi-Civita connection from this metric.

In [611]:
nab_e = g_e.connection()


In [612]:
print(nab_e)


Levi-Civita connection nabla_g_e associated with the Riemannian metric g_e on the 2-dimensional differentiable manifold N


If we use Cartesian co-ordinates, we expect that $\Gamma^k_{ij} = 0,
\forall i,j,k$. Only non-zero entries get printed.

In [613]:
print(nab_e.display(FrameCartesianN))

Just to be sure, we can print out all the entries.

In [730]:
print(nab_e[:])

[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]


In polar co-ordinates, we get

In [614]:
print(nab_e.display(FramePolarN))

Gam^r_theta,theta = -sqrt(x^2 + y^2) 
Gam^theta_r,theta = 1/sqrt(x^2 + y^2) 
Gam^theta_theta,r = 1/sqrt(x^2 + y^2) 


Which we can rew-rewrite as

$$
\begin{aligned}
\Gamma^r_{\theta,\theta} &= -r \\
\Gamma^\theta_{r,\theta} &= 1/r \\
\Gamma^\theta_{\theta,r} &= 1/r
\end{aligned}
$$

with all other entries being 0.

The Cut Sphere
==============

Our manifold is going to be the sphere with a cut from the north pole
to the south pole and we can cover this with a single chart.

In [615]:
M = Manifold(2, 'M',r'\mathcal{M}', start_index=1)

In [616]:
X.<x,y> = M.chart()

In [617]:
# X.add_restrictions([x > -pi/2 , x < pi/2, y > 0, y < 2*pi])

In [618]:
Y.<u,v> = M.chart()


In [619]:
# Y.add_restrictions([v > 0, v < 2*pi])

In [620]:
# Z.<w,z> = M.chart()

In [621]:
xy_to_uv = X.transition_map(Y, (log(tan(x/2)), y))

In [622]:
# xy_to_wz = X.transition_map(Z, (2*arctan(exp(x)), y))

In [623]:
print(xy_to_uv)

Change of coordinates from Chart (M, (x, y)) to Chart (M, (u, v))


In [624]:
# print(xy_to_wz)

In [625]:
xy_to_uv.display()

u = log(tan(1/2*x))
v = y

In [626]:
xy_to_uv.set_inverse(2*arctan(exp(u)), v)

Check of the inverse coordinate transformation:
   x == 2*arctan(sin(1/2*x)/cos(1/2*x))
   y == y
   u == u
   v == v


In [627]:
# xy_to_wz.display()

In [628]:
# xy_to_wz.set_inverse(log(tan(w/2)), z)

In [629]:
xy_to_uv.inverse().display()

x = 2*arctan(e^u)
y = v

In [630]:
# xy_to_wz.inverse().display()

In [631]:
print(X)

Chart (M, (x, y))


In [632]:
print(Y)

Chart (M, (u, v))


In [633]:
# print(Z)

In [634]:
eX = X.frame()

In [635]:
eY = Y.frame()

In [636]:
# eZ = Z.frame()

Add the metric induced from Euclidean metric in $\mathbb{R}^3$.

In [637]:
g = M.metric('g')

In [638]:
g[1,1], g[2,2] = 1, (sin(x))^2

In [639]:
print(g.display(eX))

g = dx*dx + sin(x)^2 dy*dy


As expected by the "miracle" of semi-Riemann geometry this gives a
torsion-free connection which respects the metric.



In [640]:
nab_g = g.connection()

In [641]:
print(nab_g)

Levi-Civita connection nabla_g associated with the Riemannian metric g on the 2-dimensional differentiable manifold M


In [642]:
print(nab_g.display(eX))

Gam^x_yy = -cos(x)*sin(x) 
Gam^y_xy = cos(x)/sin(x) 
Gam^y_yx = cos(x)/sin(x) 


In [643]:
print(nab_g.torsion().display(eX))

0


In [644]:
nab_g(g).display()

nabla_g(g) = 0

Create a connection with as yet no connection co-efficients.

In [645]:
nab = M.affine_connection('nabla', latex_name=r'\nabla')


In [646]:
ch_basis = M.automorphism_field()

In [647]:
ch_basis[1,1], ch_basis[2,2] = 1, sin(x)

In [648]:
e = M.default_frame().new_frame(ch_basis, 'e')

In [649]:
nab.add_coef(e)


3-indices components w.r.t. Vector frame (M, (e_1,e_2))

In [650]:
print(nab.display(e))

In [651]:
print(nab.display(eX))

Gam^y_yx = -cos(x)/sin(x) 


In [652]:
nab._coefficients

{Coordinate frame (M, (d/dx,d/dy)): 3-indices components w.r.t. Coordinate frame (M, (d/dx,d/dy)),
 Vector frame (M, (e_1,e_2)): 3-indices components w.r.t. Vector frame (M, (e_1,e_2))}

In [653]:
nab.coef()[:]

[[[0, 0], [0, 0]], [[0, 0], [-cos(x)/sin(x), 0]]]

In [654]:
nab.coef(eY)[:]

[[[2*cos(1/2*x)^2 - 1, 0], [0, 0]],
 [[0, 0], [-2*cos(1/2*x)*cos(x)*sin(1/2*x)/sin(x), 0]]]

$$
\begin{align}
{\ddot{\gamma}}^x \phantom{ + \frac{\cos x}{\sin x}\dot{\gamma}^y\dot{\gamma}^x} & = 0 \\
{\ddot{\gamma}}^y - \frac{\cos x}{\sin x}\dot{\gamma}^y\dot{\gamma}^x & = 0
\end{align}
$$

For now let us assume that the solution to ${\ddot{\gamma}}^x = 0$ is
$\gamma^x = t$. Then we can see that a solution to the second equation is
$\gamma^y = \log\tan(t/2)$.


In [655]:
print(nab.display(eY))

Gam^u_uu = 2*cos(1/2*x)^2 - 1 
Gam^v_vu = -2*cos(1/2*x)*cos(x)*sin(1/2*x)/sin(x) 


In [656]:
# print(nab.display(eZ))

$2\cos^2{x/2} - 1 = \cos{x}$ and $2\sin{x/2}\cos{x/2} = \sin{x}$ so
that $\Gamma^u_{uu} = \cos{x}$ and $\Gamma^v_{vu} = -\cos{x}$.

$$
\begin{align}
{\ddot{\gamma}}^u + \cos x\dot{\gamma}^u\dot{\gamma}^u & = 0 \\
{\ddot{\gamma}}^v - \cos x\dot{\gamma}^v\dot{\gamma}^u & = 0
\end{align}
$$


In [657]:
print(nab.torsion().display())

-cos(x)/sin(x) d/dy*dx*dy + cos(x)/sin(x) d/dy*dy*dx


In [658]:
nab(g).display()

nabla(g) = 4*cos(x)*sin(x) dy*dy*dx

The **Koszul formula** (see e.g. [@o1983semi]) characterizes the
Levi-Civita connection $\nabla$

$$
\begin{align}
2  \langle \nabla_X Y, Z\rangle & = X  \langle Y,Z\rangle + Y  \langle Z,X\rangle - Z  \langle X,Y\rangle \\
&-  \langle X,[Y,Z]\rangle +   \langle Y,[Z,X]\rangle +  \langle Z,[X,Y]\rangle
\end{align}
$$

Being more explicit about the metric, this can be re-written as

$$
\begin{align}
2 g(\nabla^g_X Y, Z) & = X g(Y,Z) + Y g(Z,X) - Z g(X,Y) \\
&- g(X,[Y,Z]) +  g(Y,[Z,X]) + g(Z,[X,Y])
\end{align}
$$


Let $\nabla^\tilde{g}$ be the Levi-Civita connection for the metric
$\tilde{g} = e^{2\sigma}g$ where $\sigma \in C^\infty M$. Following
[Gadea2010] and substituting into the Koszul formula and then applying
the product rule

$$
\begin{align}
2 e^{2 \sigma} g(\nabla^\tilde{g}_X Y, Z) & = X  e^{2 \sigma} g(Y,Z) + Y e^{2 \sigma} g(Z,X) - Z e^{2 \sigma} g(X,Y) \\
& + e^{2 \sigma} g([X,Y],Z]) - e^{2 \sigma} g([Y,Z],X) + e^{2 \sigma} g([Z,X],Y) \\
& = 2 e^{2\sigma}[g(\nabla^{g}_X Y, Z) + X\sigma g(Y,Z) + Y\sigma g(Z,X) - Z\sigma g(X,Y)] \\
& = 2 e^{2\sigma}[g(\nabla^{g}_X Y + X\sigma Y + Y\sigma X - g(X,Y) \operatorname{grad}\sigma, Z)]
\end{align}
$$


Where as usual the vector field, $\operatorname{grad}\phi$ for $\phi
\in C^\infty M$, is defined via $g(\operatorname{grad}\phi, X) =
\mathrm{d}\phi(X) = X\phi$.

In [659]:
h = M.metric('h')

In [660]:
h[1,1], h[2,2] = 1 / (sin(x))^2, 1

In [661]:
print(h.display(eX))


h = sin(x)^(-2) dx*dx + dy*dy


In [662]:
nab_h = h.connection()

In [663]:
print(nab_h)

Levi-Civita connection nabla_h associated with the Riemannian metric h on the 2-dimensional differentiable manifold M


In [664]:
print(nab_h.display(eX))

Gam^x_xx = -cos(x)/sin(x) 


In [665]:
v = M.vector_field('v')

In [666]:
v[eX,:] = [cos(x)/sin(x), 0]

In [667]:
print(v.display(eX))

v = cos(x)/sin(x) d/dx


In [668]:
print(v.display(eY))

v = 1/2*cos(x)/(cos(1/2*x)*sin(1/2*x)*sin(x)) d/du


In [669]:
print(nab_g.display())

Gam^x_yy = -cos(x)*sin(x) 
Gam^y_xy = cos(x)/sin(x) 
Gam^y_yx = cos(x)/sin(x) 


In [670]:
e_x = M.vector_field('e_x'); e_y = M.vector_field('e_y')

In [671]:
e_x[:] = [1, 0]; e_y[:] = [0, 1]

In [672]:
print(nab_g(e_x).display())

nabla_g(e_x) = cos(x)/sin(x) d/dy*dy


In [673]:
print(nab_g(e_y).display())

nabla_g(e_y) = -cos(x)*sin(x) d/dx*dy + cos(x)/sin(x) d/dy*dx


$\nabla_{\partial_i}{\partial_j} = \Gamma^k_{ij}\partial_k$

In [674]:
print(nab_g(e_y)[:])

[             0 -cos(x)*sin(x)]
[ cos(x)/sin(x)              0]


In [675]:
print(nab_g(e_x)(e_x).display())

nabla_g(e_x)(e_x) = 0


In [676]:
print(nab_g(e_y)(e_x).display())


nabla_g(e_y)(e_x) = cos(x)/sin(x) d/dy


In [677]:
print(nab_g(e_x)(e_y).display())

nabla_g(e_x)(e_y) = cos(x)/sin(x) d/dy


In [678]:
print(nab_g(e_y)(e_y).display())

nabla_g(e_y)(e_y) = -cos(x)*sin(x) d/dx


In [679]:
print(v.display())

v = cos(x)/sin(x) d/dx


In [680]:
f = M.scalar_field(-ln(sin(x)), name='f')

In [681]:
f.display()

f: M --> R
   (x, y) |--> -log(sin(x))
   (u, v) |--> -u + log(1/2*e^(2*u) + 1/2)

In [682]:
e_x(f)

Scalar field e_x(f) on the 2-dimensional differentiable manifold M

In [683]:
e_x(f).display()

e_x(f): M --> R
   (x, y) |--> -cos(x)/sin(x)
   (u, v) |--> 1/2*(e^(2*u) - 1)*e^(-u)

In [684]:
nab_g(e_x)(e_x).display()

nabla_g(e_x)(e_x) = 0

In [685]:
(e_x(f) * e_x).display()

-cos(x)/sin(x) d/dx

In [686]:
g(e_x,e_x).display()

g(e_x,e_x): M --> R
   (x, y) |--> 1
   (u, v) |--> 1

In [687]:
nab_tilde_xx = nab_g(e_x)(e_x) + e_x(f) * e_x + e_x(f) * e_x + g(e_x,e_x) * e_x * cos(x) / sin(x)

In [688]:
nab_tilde_xx.display()

-cos(x)/sin(x) d/dx

In [689]:
nab_tilde_xy = nab_g(e_y)(e_x) + e_x(f) * e_y + e_y(f) * e_x + g(e_x,e_y) * e_x * cos(x) / sin(x)

In [690]:
nab_tilde_xy.display()

0

In [691]:
nab_tilde_yx = nab_g(e_x)(e_y) + e_y(f) * e_x + e_x(f) * e_y + g(e_y,e_x) * e_x * cos(x) / sin(x)

In [692]:
nab_tilde_yx.display()

0

In [693]:
nab_tilde_yy = nab_g(e_y)(e_y) + e_y(f) * e_y + e_y(f) * e_y + g(e_y,e_y) * e_x * cos(x) / sin(x)

In [694]:
nab_tilde_yy.display()

0

In [695]:
nab_tilde = M.affine_connection('nabla_t', r'\tilde_{\nabla}')

In [696]:
print(nab_tilde)

Affine connection nabla_t on the 2-dimensional differentiable manifold M


In [697]:
for i in M.irange():
    for j in M.irange():
        for k in M.irange():
            nab_tilde.add_coef()[k,i,j] = \
                nab_g(X.frame()[i])(X.frame()[j])(X.coframe()[k]) + \
                X.frame()[i](f) * X.frame()[j](X.coframe()[k]) + \
                X.frame()[j](f) * X.frame()[i](X.coframe()[k]) + \
                g(X.frame()[i], X.frame()[j]) * X.frame()[1](X.coframe()[k]) * cos(x) / sin(x)

In [698]:
nab_tilde.display()

Gam^x_xx = -cos(x)/sin(x) 

In [699]:
print(nab_tilde.display())

Gam^x_xx = -cos(x)/sin(x) 


In [700]:
nab_g.display()

Gam^x_yy = -cos(x)*sin(x) 
Gam^y_xy = cos(x)/sin(x) 
Gam^y_yx = cos(x)/sin(x) 

In [701]:
print(g(X.frame()[1],X.frame()[1]))

Scalar field g(d/dx,d/dx) on the 2-dimensional differentiable manifold M


In [702]:
print(x)

x


In [703]:
print(nab_g(X.frame()[1])(X.frame()[1])(X.coframe()[1]))

Scalar field dx(nabla_g(d/dx)(d/dx)) on the 2-dimensional differentiable manifold M


In [704]:
foo = nab_g(X.frame()[1])(X.frame()[1])(X.coframe()[1])

In [705]:
foo.display()

dx(nabla_g(d/dx)(d/dx)): M --> R
   (x, y) |--> 0
   (u, v) |--> 0

In [706]:
dx = M.default_frame().coframe()

In [707]:
check = []

In [708]:
for i in M.irange():
    for j in M.irange():
            check.append( nab.connection_form(i,j) == \
                          sum( nab[[i,j,k]]*dx[k] for k in M.irange() ) )

In [709]:
check

[True, True, True, True]